### Proof of Concept for Waze Stuff 


In [1]:
from shapely.geometry import Point,shape, mapping
from IPython.display import clear_output
from shapely.ops import unary_union
import fiona
from fiona import collection

C:\Users\Coleman.Shepard.ctr\AppData\Local\Continuum\anaconda3\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


### Load in shapefiles, buffer roads, and union the buffers

In [5]:
accidents = fiona.open('HPMS/ggez.shp', 'r')
roadway = fiona.open('Alabama_Count.shp', 'r')

In [9]:
%%time

y = 0

with fiona.open('Terry_09122019_156.shp', 'w', driver='ESRI Shapefile', schema=roadway.schema) as output:
    # Loop through road shapefile and review geometry
    for road in roadway:
        y+=1

        clear_output()
        print('Gone through', y, 'records out of 172,674 aka', round((y/172674)*100,2), 'percent')

        # There are some records with a NoneType for geometry, but this can be passed over
        if road['geometry'] != None:
            geom = shape(road['geometry']).buffer(0.001)

            for accident in accidents:
                lat = accident['geometry']['coordinates'][1]
                lon = accident['geometry']['coordinates'][0]
                bounds = roadway.bounds

                # filter based on bounding box of roads
                if lat > bounds[1] and lat < bounds[3]:
                    if lon > bounds[0] and lon < bounds[2]:
                        if geom.contains(shape(accident['geometry'])) == True:
                            road['properties']['Count'] += 1

            output.write(road)

Gone through 172674 records out of 172,674 aka 100.0 percent
Wall time: 1h 52min 34s
